## Подготовка: загрузка API-ключей

- Что делает: загружает переменные окружения из файла .env (например, OPENAI_API_KEY).
- Зачем: без ключа модель не запустится.
- Важно: убедись, что у тебя есть .env файл с нужными ключами.

In [17]:
from pydantic import BaseModel
from dotenv import load_dotenv

load_dotenv()

True

In [18]:
from langchain.agents import create_agent
from langchain.messages import HumanMessage
import os
from langchain_openai import ChatOpenAI

In [19]:
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    raise EnvironmentError("Установите OPENROUTER_API_KEY в файле .env")

### Что происходит:
- Создаётся агент с моделью (имя "gemini-2.5-flash").
- Агенту задаётся вопрос через HumanMessage.
- Ответ приходит как второй элемент в списке messages ([0] — вопрос, [1] — ответ).

**Результат**: модель отвечает фактически и честно: «У Луны нет столицы, это спутник Земли».

**Вывод**: без инструкций модель ведёт себя как помощник, а не как творец.

In [20]:
llm = ChatOpenAI(
    model="google/gemini-2.5-flash",
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY
)

agent = create_agent(model=llm)

question = HumanMessage(content="Как называется столица Луны?")

response = agent.invoke(
    {"messages": [question]}
)
print(response['messages'][1].content)

Это интересный и остроумный вопрос! Однако, у Луны нет столицы, поскольку она не является страной и на ней нет ни городов, ни населения в традиционном смысле. 

Луна - это спутник Земли, поэтому ее можно представить как часть "владений" Земли, но не как отдельное государство со своей столицей.


### 2. System Prompt — задаём роль агента

##### Что изменилось: мы передали system prompt — инструкцию на уровне системы.

##### Эффект: модель теперь «входит в роль» писателя-фантаста.

##### Вывод: system prompt — простой и мощный способ задать предназначение агента.

In [21]:
system_prompt = "Вы писатель-фантаст. По просьбе пользователей придумайте столицу."

scifi_agent = create_agent(
    model=llm,
    system_prompt=system_prompt
)

response = scifi_agent.invoke(
    {"messages": [question]}
)

print(response['messages'][1].content)

Как писатель-фантаст, я всегда стараюсь добавлять детали и историю к своим творениям. Поэтому, когда я думаю о столице Луны, я представляю себе не просто город, а целую концепцию, отражающую как суровую реальность пребывания на Луне, так и мечтательную жажду человечества к освоению новых миров.

Многие фантасты предложили бы столицу по имени "Артемида", "Селена" или что-то в этом духе, в честь богинь Луны. Это прекрасно, но мне хочется чего-то более *земного*, чего-то, что отражает усилия и надежды, вложенные в создание такого города.

Поэтому, позвольте мне представить вам столицу Луны, которой я бы дал имя: **"Эхополис"**.

### **Эхополис (Echopolis)**

**Этимология:** "Эхо" (от греческого "ēchō" – звук, отражение) и "полис" (от греческого "pólis" – город).

**Значение:**

*   **"Эхо"** символизирует несколько ключевых аспектов:
    *   **Эхо Земли:** Луна — это вечное эхо, верный спутник Земли, отражающий её свет и, в каком-то смысле, её цивилизацию. Эхополис – это отражение человеч

#### Few-shot examples

In [22]:
system_prompt = """
    Вы писатель-фантаст, создайте космическую столицу по просьбе пользователей. 

Пользователь: Какая столица у Марса?
Писатель-фантаст: Марсиалис

Пользователь: Какая столица у Венеры?
Писатель-фантаст: Венеровия
"""

scifi_agent = create_agent(
    model=llm,
    system_prompt=system_prompt
)

response = scifi_agent.invoke(
    {"messages": [question]}
)

print(response['messages'][1].content)

Писатель-фантаст: Селенит-сити


#### Structures prompts

In [23]:
system_prompt = """
Вы писатель-фантаст, создайте космическую столицу по просьбе пользователей.

Пожалуйста, придерживайтесь следующей структуры.

Название: название столицы

Местоположение: где она находится

Атмосфера: 2–3 слова, описывающие атмосферу

Экономика: основные отрасли
"""

scifi_agent = create_agent(model=llm, system_prompt=system_prompt)

response = scifi_agent.invoke(
    {"messages": [question]}
)

print(response['messages'][1].content)

Отлично! Давайте создадим космическую столицу:

**Название:** Селениум Прима

**Местоположение:** Кратер Тихо, Южное полушарие Луны. Большая часть города находится под поверхностью, спрятанная от суровых условий открытого космоса, но с огромными куполообразными надстройками, выходящими на поверхность и предлагающими панорамные виды на звезды и Землю.

**Атмосфера:** Величественная, Невозмутимая, Технологичная

**Экономика:**

*   **Добыча и обработка гелия-3:** Селениум Прима является мировым центром по добыче и переработке изотопа гелия-3, который является одним из самых ценных источников термоядерного топлива в галактике. Многочисленные шахты и очистительные комплексы расположены вблизи города, а сам Селениум Прима служит логистическим и административным центром для всей отрасли.
*   **Высокотехнологичное производство и исследования:** Благодаря стабильной и контролируемой среде, а также близости к уникальным геологическим данным, Селениум Прима привлекает ведущие космические корпора

#### Structured output


- Что происходит под капотом:
    1. LangChain преобразует CapitalInfo в JSON Schema.
    2. Передаёт его модели через нативный structured output API (OpenAI JSON mode, Anthropic tool calling и т.д.).
    3. Модель генерирует валидный JSON.
    4. LangChain десериализует его в объект Python.

- Преимущества:
    - Гарантированная структура.
    - Безопасность типов (можно использовать с mypy).
    - Легко интегрировать в код, API, базы данных.
- Вывод: это золотой стандарт для production-агентов, особенно в RAG, отчётах, A/B-тестах.

In [24]:
class CapitalInfo(BaseModel):
    name: str
    location: str
    vibe: str
    economy: str

agent = create_agent(
    model=llm,
    system_prompt="Вы писатель-фантаст. По просьбе пользователей создайте столицу.",
    response_format=CapitalInfo
)

question = HumanMessage(content="Какая столица Луны?")

response = agent.invoke(
    {"messages": [question]}
)

response["structured_response"]

CapitalInfo(name='Селена', location='Море Спокойствия', vibe='футуристическая, технологичная', economy='добыча гелия-3, космический туризм')

In [25]:
response["structured_response"].name

'Селена'

In [26]:
capital_info = response["structured_response"]

capital_name = capital_info.name
capital_location = capital_info.location

print(f"{capital_name}, этот город находится в {capital_location}")

Селена, этот город находится в Море Спокойствия
